In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [6]:
import addict
import copy
import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

import celeri_closure
celeri_closure = reload(celeri_closure)
import celeri
celeri = reload(celeri)

In [10]:
%%time
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
#command_file_name = './data/global/global_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure = celeri.assign_block_labels(segment, station, block, mogi, sar)

CPU times: user 955 ms, sys: 44.1 ms, total: 999 ms
Wall time: 998 ms


In [8]:
celeri.plot_block_labels(segment, block, station, closure)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
vertices = closure.polygon_vertices[6]

In [79]:
%%time
intervals = []
for i in range(vertices.shape[0] - 1):
    s1 = vertices[i][0]
    s2 = vertices[i+1][0]
    
    # If the longitudes are separated by more than 180 degrees, then 
    # this is a meridian crossing segment where s1 is <180 and s2 is
    # >180 degrees. So use (s1, 0) and (360, s2).
    if s2 - s1 > 180:
        intervals.append((0, s1))
        intervals.append((s2, 360))
    # Similiarly, separated by -180 degrees suggests that s1 is >180
    # and s2 is <180. So use (s1,360), (0,s2)
    elif s2 - s1 < -180:
        intervals.append((s1, 360))
        intervals.append((0, s2))
    else:
        intervals.append((s1, s2))
intervals = np.array([(s1, s2) if s1 < s2 else (s2, s1) for s1, s2 in intervals])

CPU times: user 72 µs, sys: 0 ns, total: 72 µs
Wall time: 76.1 µs


In [80]:
intervals

array([[215.06 , 220.823],
       [211.394, 215.06 ],
       [205.239, 211.394],
       [191.226, 205.239],
       [179.964, 191.226],
       [163.856, 179.964],
       [145.025, 163.856],
       [142.758, 145.025],
       [131.169, 142.758],
       [124.115, 131.169],
       [124.115, 164.764],
       [164.764, 180.29 ],
       [162.871, 180.29 ],
       [162.871, 204.097],
       [204.097, 252.514],
       [252.514, 259.341],
       [259.341, 260.639],
       [260.639, 274.413],
       [274.413, 325.661],
       [272.027, 325.661],
       [223.835, 272.027],
       [220.823, 223.835]])

In [81]:
%%time
# Fun classic intro algorithms problem: how to combine intervals in O(n log(n))?
# Sort them by the first value, and then combine adjacent intervals. 
sorted_intervals = intervals[intervals[:,0].argsort()]
combined_intervals = []
cur_interval = sorted_intervals[0]
for I in sorted_intervals[1:]:
    if I[0] > cur_interval[1]:
        print(cur_interval, I)
        combined_intervals.append(cur_interval)
        cur_interval = I
    else:
        cur_interval[1] = max(cur_interval[1], I[1])
combined_intervals.append(cur_interval)

CPU times: user 236 µs, sys: 0 ns, total: 236 µs
Wall time: 204 µs


In [82]:
combined_intervals

[array([124.115, 325.661])]

In [5]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)

Isolating slip rate constraints
- Tensile-slip rate constraint on 11ba: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-1.00 (mm/yr)
- Tensile-slip rate constraint on 11bb: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-1.00 (mm/yr)
- Dip-slip rate constraint on AN_Iran_fault_2: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-3.00 (mm/yr)
- Dip-slip rate constraint on AN_Iran_fault_6: rate = 1.50 (mm/yr), 1-sigma uncertainty = +/-1.00 (mm/yr)
- Tensile-slip rate constraint on AWAT_fault_3: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-5.00 (mm/yr)
- Strike-slip rate constraint on Akar_connect2: rate = -8.00 (mm/yr), 1-sigma uncertainty = +/-5.00 (mm/yr)
- Dip-slip rate constraint on BOOB_fault_1: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-10.00 (mm/yr)
- Dip-slip rate constraint on BOOB_fault_4: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-10.00 (mm/yr)
- Tensile-slip rate constraint on BTFZ_Baldwin12b: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-10.00 (mm/yr)
- Tensile-slip rate cons

In [6]:
command = addict.Dict(command)